# Notebook pour transforemr des fichiers excel en modeles django


In [288]:
!pip install jr_data_science

distributed 1.21.8 requires msgpack, which is not installed.


In [289]:
import jr_data_science as ds
from jr_data_science import functions_de_stats_sur_df
from jr_data_science import useful_functions
from jr_data_science import functions_de_decouverte_de_fichiers

In [290]:
import pandas as pd

In [291]:
def description(df):
    sep = "|"
    for col in df.columns:
        print (sep.join([str(x) for x in [col, df[col].nunique(), 1 - df[col].isnull().sum()/df.shape[0]]]))

In [381]:

def get_connection(path_sqlite3):
    import sqlite3
    conn = sqlite3.connect(path_sqlite3)
    return conn

def excel_to_sqlite(path_xls, path_sqlite3, table_name, if_exists="replace"):
    import pandas as pd
    df = pd.read_excel(path_xls)
    df_to_sqlite(df, path_sqlite3, table_name, if_exists)
        
def df_to_sqlite(df, path_sqlite3, table_name, if_exists="replace"):
    conn = get_connection(path_sqlite3)
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)

In [382]:
from jr_data_science import functions_pour_analyser_des_df

In [383]:
def clean_string(string):
    string = string.replace("é", "e")
    string = string.replace("è", "e")
    string = string.replace(" ", "_")
    string = string.replace("€", "_eur")    

    a = "".join([x.lower()  for x in string if x.lower() in "_azertyuiopqsdfghjklmwxcvbn1234567890".lower()])
    a = a.replace("__", "_").replace("__", "_")
    a = a[:-1] if a.endswith("_") else a
    return a

In [384]:
def  write_df_to_django_model(output_file, df, nom_df="nom_df"):
    create_dir_if_not_exit(output_file)
    with open(output_file, "w") as f:
        f.writelines(df_to_django_model(df, nom_df))

In [660]:
def df_to_django_model(df, nom_df="nom_df", debug=False):
    """
    Output on screen a proposal of a django model.
    """
    conversion = {  "datetime64[ns]" : "models.DateTimeField(blank=False, null=True)",
                    "int64"          : "models.IntegerField(blank=True, null=True)"   ,
                    "object"         : "models.TextField(blank=True, null=True)"      ,
                    "float64"        : "models.FloatField(blank=True, null=True)"     }
    max_len = max([len(x) for x in df.columns])
    nb_car  = max_len + 2
    tab      = " "*4
    to_print ="""
from django.db 						import models
from django.contrib.auth.models 	import User
from django.forms 					import ModelForm
from django.urls import reverse
"""
    to_print +="class %s (models.Model):\n"%nom_df
    for col, dtype in zip(df.columns, df.dtypes):
        blank = nb_car - len(col)
        try:
            to_print +="%s%s %s= %s\n"%(tab, col, " "*blank, conversion[dtype.name])
        except:
            to_print +="%s%s %s= %s\n"%(tab, col, " "*blank, conversion["object"])
    if debug : print(to_print)
        
    to_print +="%sdef __str__(self):\n"%(tab)
    to_print +="%s%sr = ''\n"%(tab, tab)
    for col  in df.columns[:3]:
        to_print +="%s%sr+= str(self.%s)\n"%(tab,tab, col)
    to_print +="%s%sreturn r"%(tab,tab)
    
    return to_print


In [638]:
def write_auto_form(output_file, nom_model, df):
    create_dir_if_not_exit(output_file)
    with open(output_file, "w") as f:
        f.writelines(df_to_auto_form(nom_model, df))

In [639]:
def df_to_auto_form(nom_model, df, debug=False):
    to_print =""
    fields = "                   ".join( ["""  "%s",\n"""%x for x in df.columns] )
    to_print = """    
from .models import *
from django.db import models
from django.forms import ModelForm

class {model}Form(ModelForm):
    error_css_class     = 'error'
    required_css_class  = 'required'
    class Meta:
        model   = {model}
        fields = [ {fields} ]
                   
    def __init__(self, *args, **kwargs):
        super({model}Form, self).__init__(*args, **kwargs)  # Call to ModelForm constructor
        height  = 1
        width   = 40
        for to_change in self.fields:
            self.fields[to_change ].widget.attrs['cols'] = width
            self.fields[to_change ].widget.attrs['rows'] = height


""".format(model= nom_model,  fields=fields)
    if debug : print(to_print)
    for action in ["Create", "Detail", "Update", "Delete", "List"]:
        to_print += """
class {action}{model}Form({model}Form):
    def __init__(self, *args, **kwargs):
        super({action}{model}Form, self).__init__(*args, **kwargs)  # Call to ModelForm constructor
    """.format(model= nom_model, action=action, fields=fields)
    if debug : print(to_print)
    return to_print

In [640]:
def write_auto_views(output_dir, nom_model):
    """
    Ecrit un fichier de vue automatiques pour django.
    """
    import os
    create_dir_if_not_exit(os.path.join(output_dir, "touch"))
    to_write = df_to_auto_views(nom_model)
    with open(os.path.join(output_dir, "views.py"), "w") as f:
        f.writelines(to_write)

In [776]:
def df_to_auto_views(nom_model, debug=False):
    to_print = """
from django.views.generic           import ListView
from django.views.generic           import UpdateView
from django.views.generic           import FormView
from django.views.generic           import DetailView
from django.views.generic           import DeleteView
from django.views.generic           import CreateView
from django.urls                    import reverse_lazy
from django.contrib.auth.decorators import login_required
from django.utils.decorators        import method_decorator
from .models import *
from .forms import *

    """
    for action in ["Create", "Detail", "Update", "Delete", "List"]:
        to_print += """
class {model}{action}({action}View):
    model                = {model}
    form_class           = {action}{model}Form
    template_name_suffix = '_{action_small}'
    """.format(model= nom_model, action=action, action_small=action.lower())
        if action in ["Delete"]:
            to_print +="""
    success_url          = reverse_lazy('List{model}')
    """.format(model=nom_model)
    if debug : print(to_print)
    return to_print

In [642]:
def create_dir_if_not_exit(filepath):
    import os
    os.makedirs(os.path.dirname(filepath), exist_ok=True)


In [643]:
def write_url(output_path, model):
    """
    Ecrit les urls django dans un fichier output (call creer urls)
    
    output_path : chemin complet vers le fichier cible
    Return :
        Void.
    """
    create_dir_if_not_exit(output_path)
    with open(output_path, "w") as f:
        f.writelines(creer_urls(model))
        

In [644]:
def creer_urls(model, debug=False):
    """
    Créer une chaine de caractere avec les urls django pour les vues auto d'une app.
    
    Return : 
        string
    """
    to_write = """
    
from django.urls import path
from django.contrib.auth.decorators import login_required
from django.urls import path

from .views import *
from django.contrib.auth.decorators import login_required
from django.urls import path

urlpatterns = [ path('Create{app}/'            , login_required({app}Create.as_view())   ,  name="Creer{app}"    ),
                 path('List{app}/'              , login_required({app}List.as_view())     , name="List{app}"),
                 path('Update{app}/<int:pk>'    , login_required({app}Update.as_view())   , name="Update{app}"   ),
                 path('Detail{app}/<int:pk>'    , login_required({app}Detail.as_view())   , name="Detail{app}"   ),
                 path('Delete{app}/<int:pk>'    , login_required({app}Delete.as_view())   , name="Delete{app}"   ),]

""".format(app=model)
    if debug : print(to_write)
    return to_write


In [694]:
def get_upper_dir(dirpath):
    import os
    if dirpath.endswith("/"):
        dirpath = dirpath[:-1]        
    upper_dir = os.path.dirname(dirpath)
    return upper_dir


In [725]:
def get_pk_from_template_dir(template_dir):
    import os
    app_dir = get_upper_dir(get_upper_dir(template_dir))
    model_path = os.path.join(app_dir, "models.py")
    return get_pk_from_modelfile(model_path)

In [726]:
def get_pk_from_modelfile(model_file_path):
    with open(model_file_path, "r") as f:
        r = [ x for x in f.readlines() if  "primary_key" in x ]
    return r[0].split()[0]
    

In [716]:
get_pk_from_template_dir("/Users/romain/PycharmProjects/CCVS_reloaded/Fichier_des_adherents/templates/")

'n_adherent'

In [801]:
def create_template(template_dir, name_model, debug=False):

    urls = {
        "Create" : "Creer" , 
        "List"  : "List", 
        "Update" : "Update", 
        "Detail"  : "Detail", 
        "Delete" : "Delete", 
        "confirm_delete" : "", 
        "Form" : "Creer"
    }
    pk0 = str(get_pk_from_template_dir(template_dir))
    
    for action in ["Create", "List", "Update", "Detail", "Delete", "confirm_delete", "Form"]:
        path_ = template_dir  + name_model + "_"+  action + ".html"
        path_ = path_.lower()
        create_dir_if_not_exit(path_)
        bof = urls[action]
        pk = "pk=object.%s"%pk0 if bof not in ["Creer", "List"] else ""
        url = bof+name_model
        if debug : print("Template into %s"%path_)
        if bof in ["Creer"]:
            write_template_creer(path_, url, pk)
        elif bof in ["List"]:
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_list(path_, url, pk)
        elif bof in ["Update"]:
            url = "Delete"+name_model
            pk = "pk=object.%s"%pk0
            write_template_update(path_, url, pk)
        elif bof in ["Detail"]:        
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_detail(path_, url, pk)
        elif bof in ["Delete"]:    
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_delete(path_, url, pk)
        else:
            write_template_display(path_, url, pk)
            


        

In [786]:
def write_template_delete(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
    
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<form method="post">{% csrf_token %}
    <p>Are you sure you want to delete "{{ object }}"?</p>
    <input type="submit" value="Confirm" />
</form>
        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a> <br>
        {% endblock %}
                """
            f.writelines(to_write)

In [787]:
def write_template_list(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
{% extends "base.html" %}
{% block content %}
    <div widt="70%">
         {% for object in object_list %}
            
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a> <br>
         {% endfor %}        
    </div>


{% endblock %}
        """
            f.writelines(to_write)

In [788]:
def write_template_creer(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<table>
        <th colspan="2">
        <h4>Créer </h4>
</th>
        <form method="post">{% csrf_token %}
            {{form.as_table}}
            <tr>
            <td>
</td>
<td>
            <input type="submit" value="Créer" class="btn btn-primary" />
</td>
</tr>
</table>
        {% endblock %}
        """
            f.writelines(to_write)

In [800]:
def write_template_update(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<table>
        <th colspan="2">
        <h4>Mise à jour </h4>
</th>
        <form method="post">{% csrf_token %}
            {{form.as_table}}
            <tr>
            <td>
</td>
<td>
            <input type="submit" value="Mettre à jour" class="btn btn-primary" />
            </form>
</td>
</tr>
       <tr>
            <td>
</td>
<td>
            <a href={% url '"""+ url + """' """ + pk + """ %}> Delete </a>
</td>
</tr>
</table>
        {% endblock %}
        """
            f.writelines(to_write)

In [790]:
def write_template_unique(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
        <table>
            
                {{object}}
            <tr>
                <td>
                </td>
                <td>
                </td>
            </tr>
        </table>
        </div>
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} > Modifier {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [791]:
def write_template_detail(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">

             {{object}}<br>

        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >Update {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [792]:
def write_template_display(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">

             {{object}}<br>

        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [793]:
def write_template_unique(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
        <table>
            
                {{object}}
            <tr>
                <td>
                </td>
                <td>
                </td>
            </tr>
        </table>
        </div>
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} > Modifier {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [794]:
def write_to_csv(csv_path, df, debug=False):
    create_dir_if_not_exit(csv_path)    
    if debug : print("going to write to {path}".format(path=csv_path))
    df.to_csv(csv_path, encoding="UTF-8", index=False, sep="|")

In [795]:
def get_connection(path_sqlite3):
    import sqlite3
    conn = sqlite3.connect(path_sqlite3)
    return conn
def get_df_from_sqlite_table(path_sqlite3, table_name):
    import pandas as pd
    cnx = get_connection(path_sqlite3)
    df = pd.read_sql_query("SELECT * FROM %s"%table_name, cnx)
    return df

def excel_to_sqlite(path_xls, path_sqlite3, table_name, if_exists="replace"):
    import pandas as pd
    df = pd.read_excel(path_xls)
    df_to_sqlite(df, path_sqlite3, table_name, if_exists)
        
def df_to_sqlite(df, path_sqlite3, table_name, if_exists="replace"):
    conn = get_connection(path_sqlite3)
    df.to_sql(table_name, conn, if_exists=if_exists)

# USAGE

In [796]:
dir_ = "./../../zz_non_app/data/pour_romain/"
paths = functions_de_decouverte_de_fichiers.get_all_specific_paths_under_dir(dir_, "xlsx" )

In [797]:
paths

['./../../zz_non_app/data/pour_romain/sommaires_revue/Ecriture.xlsx',
 './../../zz_non_app/data/pour_romain/sommaires_revue/Details_des_articles.xlsx',
 './../../zz_non_app/data/pour_romain/sommaires_revue/Revues.xlsx',
 './../../zz_non_app/data/pour_romain/sommaires_revue/Auteurs_details.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Situation_de_la_collection.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Fiches_evenements.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Collections_Archives_mai_2006.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Cotisations.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Cotisations_des_adherents.xlsx',
 './../../zz_non_app/data/pour_romain/adherents/Fichier_des_adherents.xlsx',
 './../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnes_HetPl.xlsx',
 './../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnements_Renouvellements_HetPl.xlsx',
 './../../zz_non_app/data/pour_romain/homme_et_plantes/Fournisseur

In [842]:
base_dir = "/Users/romain/PycharmProjects/CCVS_reloaded"
path_sqlite3 = os.path.join(base_dir, "db.sqlite3")
for path in paths:
    #print (path)
    nom        = functions_de_decouverte_de_fichiers.get_file_name(path)
    df         = pd.read_excel(path)
    df.columns = [clean_string(x) for x in df.columns]
    model      = nom[:nom.find(".")]
    nom_app    = model
    lister_les_urls(model)
    #write_df_to_django_model("{base_dir}/{nom_app}/models.py".format(base_dir=base_dir, nom_app=nom_app), df, nom_app)
    #write_auto_views("{base_dir}/{nom_app}".format(base_dir=base_dir, nom_app=nom_app), model)
    #create_template("{base_dir}/{nom_app}/templates/{nom_app}/".format(base_dir=base_dir, nom_app=nom_app), model)
    #write_url("{base_dir}/{nom_app}/urls.py".format(base_dir=base_dir, nom_app=nom_app), model)    
    #write_auto_form("{base_dir}/{nom_app}/forms.py".format(base_dir=base_dir, nom_app=nom_app),model, df)
    #write_to_csv("{base_dir}/{nom_app}/data/{nom_app}.csv".format(base_dir=base_dir, nom_app=nom_app), df, True)

    #with open(os.path.join(base_dir, os.path.join(os.path.basename(base_dir), "urls.py")), "a") as f:
    #    f.write("\nurlpatterns += [path('{nom_app}/', include('{nom_app}.urls'),)]".format(nom_app=nom_app))
    #with open(os.path.join(base_dir, os.path.join(os.path.basename(base_dir), "settings.py")), "a") as f:
    #    f.write("\nINSTALLED_APPS += [ '{nom_app}']".format(nom_app=nom_app))


<tr><td>Ecriture</br>
<a href={% url 'CreerEcriture' %}>CreerEcriture</a>
<a href={% url 'ListEcriture' %}>ListEcriture</a>
</td></tr>
<tr><td>Details_des_articles</br>
<a href={% url 'CreerDetails_des_articles' %}>CreerDetails_des_articles</a>
<a href={% url 'ListDetails_des_articles' %}>ListDetails_des_articles</a>
</td></tr>
<tr><td>Revues</br>
<a href={% url 'CreerRevues' %}>CreerRevues</a>
<a href={% url 'ListRevues' %}>ListRevues</a>
</td></tr>
<tr><td>Auteurs_details</br>
<a href={% url 'CreerAuteurs_details' %}>CreerAuteurs_details</a>
<a href={% url 'ListAuteurs_details' %}>ListAuteurs_details</a>
</td></tr>
<tr><td>Situation_de_la_collection</br>
<a href={% url 'CreerSituation_de_la_collection' %}>CreerSituation_de_la_collection</a>
<a href={% url 'ListSituation_de_la_collection' %}>ListSituation_de_la_collection</a>
</td></tr>
<tr><td>Fiches_evenements</br>
<a href={% url 'CreerFiches_evenements' %}>CreerFiches_evenements</a>
<a href={% url 'ListFiches_evenements' %}>ListFic

In [841]:
def lister_les_urls(app):
    actions = ["Creer","List"]#,"Update","Detail","Delete",]
    print("<tr><td>{app}</br>".format(app=app))
    for action in actions:
        print("<a href={{% url '{action}{app}' %}}>{action}{app}</a>".format(action=action, app=app))
    print("</td></tr>")

        


# Une fois ici, il faut définir les contraintes sur les tables dans les fichiers models.py des app
# ensuite pour insérer les df dans les tables, il faut ajouter les mots "id" à la fin des colonnes des champs qui ont été référencés comme des foreignkey dans les models
# c'est cette dernière étape qu'on essaie ici

In [508]:
paths = [
"./../../zz_non_app/data/pour_romain/adherents/Cotisations.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Fichier_des_adherents.xlsx",
"./../../zz_non_app/data/pour_romain/sommaires_revue/Auteurs_details.xlsx",

"./../../zz_non_app/data/pour_romain/sommaires_revue/Revues.xlsx",

"./../../zz_non_app/data/pour_romain/sommaires_revue/Details_des_articles.xlsx",


"./../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnes_HetPl.xlsx",
"./../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnements_Renouvellements_HetPl.xlsx",
"./../../zz_non_app/data/pour_romain/homme_et_plantes/Fournisseurs_d_abonnements.xlsx",

"./../../zz_non_app/data/pour_romain/adherents/Fiches_evenements.xlsx",
"./../../zz_non_app/data/pour_romain/sommaires_revue/Ecriture.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Cotisations_des_adherents.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Situation_de_la_collection.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Collections_Archives_mai_2006.xlsx",
]

In [392]:
for path in paths:
    nom        = functions_de_decouverte_de_fichiers.get_file_name(path)
    df         = pd.read_excel(path)
    df.columns = [clean_string(x) for x in df.columns]
    model      = nom[:nom.find(".")]
    nom_app    = model
    table_name="{appname}_{appname_lower}".format(appname=nom_app,appname_lower=nom_app.lower())
    print(table_name, df.shape)
    #df_to_sqlite(df, path_sqlite3, table_name)
    


Fichier_des_adherents_fichier_des_adherents (303, 29)
Auteurs_details_auteurs_details (468, 10)
Cotisations_cotisations (9, 4)
Revues_revues (102, 9)
Details_des_articles_details_des_articles (916, 11)
Abonnes_HetPl_abonnes_hetpl (1284, 26)
Abonnements_Renouvellements_HetPl_abonnements_renouvellements_hetpl (16845, 11)
Fournisseurs_d_abonnements_fournisseurs_d_abonnements (111, 5)
Fiches_evenements_fiches_evenements (129, 10)
Ecriture_ecriture (1072, 3)
Cotisations_des_adherents_cotisations_des_adherents (4714, 18)
Situation_de_la_collection_situation_de_la_collection (7, 3)
Collections_Archives_mai_2006_collections_archives_mai_2006 (368, 35)


In [621]:
path = paths[13]
path

IndexError: list index out of range

In [614]:
keys = ["label", "n_adherent", ]

In [615]:
df = pd.read_excel(path)
df.columns = [clean_string(x) for x in df.columns]
df.columns = ["%s_id"%x if x in keys else x for x in df.columns ]
df.head(2).T

,0,1
n,1,2
nom_de_la_collection,Cornus,Peperomia
nom_vernaculaire,Cornouiller,NaN
annee_de_labellisation,1990-12-01 00:00:00,2001-01-01 00:00:00
label_id,CN,CN
proprietaire,UNIVERSITE LOUIS PASTEUR (JARDIN BOTANIQUE),VILLE DE CAEN (JARDIN BOTANIQUE)
n_adherent_id,279,276
responsable,Jacques Zeller,Guy Madelaine
adresse,28 rue Goethe,5 place Blot
code_postal,67083,14000


In [616]:
table_name = os.path.basename(path)
table_name = table_name[:table_name.find(".")]
table_name = "%s_%s"%(table_name, table_name.lower())
table_name

'Collections_Archives_mai_2006_collections_archives_mai_2006'

In [617]:
df.shape

(368, 35)

In [620]:
df_to_sqlite(df, path_sqlite3, table_name, "append")